# Big Data Analytics: NYC Crashes

### Installation der Python Packages

In [1]:
pip install kafka-python pymongo

Note: you may need to restart the kernel to use updated packages.


In [6]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

producer.send('crashes', value=bytearray('Hello, World!!!', encoding='utf-8'), key=bytearray(str(99), encoding='utf-8'))


KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.

In [ ]:
from kafka import KafkaConsumer
from pymongo import MongoClient
import datetime as dt

consumer = KafkaConsumer('crashes', bootstrap_servers=['localhost:9092'])

client = MongoClient("")

yellow_collection = client['big-data']['crashes']
yellow_collection.delete_many({})

count = 0

for msg in consumer: 
    count += 1
    print('Received new message: %s' % count)
    values = msg.value.decode('utf-8').split(',')